## Generative pretrained image priors for MRI reconstruction
**Creator**: [Guanxiong Luo](https://ggluo.github.io)

**Overview**:
This notebook is to present how to use the pretrained image priors to improve MR reconstruction. What we will use here are the following:
1. a library for generative image prior, called [spreco](https://pypi.org/project/spreco),
2. a MR image reconstruction toolbox, called [BART](https://www.github.com/mrirecon/bart),
3. several pretrained models hosted on this huggingface [page](https://huggingface.co/Guanxiong/MRI-Image-Priors).



#### Let's get all of them at first!

In [ ]:
%%bash
git clone https://github.com/ggluo/spreco
cd spreco
git checkout devel
pip install .
pip install huggingface_hub


#### Prepare BART

In [ ]:
%%bash

# Install BARTs dependencies
apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

# Download BART version
[ -d /content/bart ] && rm -r /content/bart
git clone https://github.com/mrirecon/bart/ bart
[ -d "bart" ] && echo "BART was downloaded successfully."

cd bart
make &> /dev/null

In [ ]:
%env TOOLBOX_PATH=/content/bart

In [ ]:
import os
os.environ['PATH'] = os.environ['TOOLBOX_PATH'] + ":" + os.environ['PATH']

#### Download models

In [ ]:
from huggingface_hub import hf_hub_download
files=["config.yaml", "smld.data-00000-of-00001", "smld.index", "smld.meta"]
for file in files:
    hf_hub_download(repo_id="Guanxiong/MRI-Image-Priors", subfolder="Diffusion/SMLD", filename=file, local_dir="./")

#### Create a function to plot the grid of images

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from functools import partial

def subplot(ax, img, title, cmap, interpolation, vmin, vmax):
    ax.imshow(img, cmap=cmap, interpolation=interpolation, vmin=vmin, vmax=vmax)
    ax.set_title(title)
    ax.axis('off')


plot_params = {'cmap': 'gray', 'interpolation': 'none', 'vmin': 0}
axplot      = partial(subplot, **plot_params)

#### Generate MR images with the downloaded model

In [ ]:
from spreco.common import utils
from spreco.sampler import sampler
import numpy as np

log_path   = './Diffusion/SMLD'
model_path = log_path+'/smld'
config     = utils.load_config(log_path+'/config.yaml')

N=100
sigma_type='quad'
target_snr=0.2
print("INFO -> sigma type: %s, sigma max: %.4f, simga min: %.4f, discrete steps: %d "%( sigma_type, config['sigma_max'], config['sigma_min'], N))

config['N'] = N
a_sampler = sampler(config, target_snr, sigma_type)
a_sampler.init_sampler(model_path, gpu_id='0')

In [ ]:
grid_x=5
grid_y=5
samples_n, samples  = a_sampler.pc_sampler(grid_x*grid_y, 3)

images = samples_n[-1]
images = np.abs(utils.float2cplx(images))

fig, axss = plt.subplots(grid_x, grid_y, figsize=(10, 10), gridspec_kw={'width_ratios': [1  for _ in range(grid_x)]})
for i in range(grid_x):
    for j in range(grid_y):
        if i==0:
            strs='x_%d'%j
        else:
            strs=''
        axplot(axss[i,j], images[i*grid_y+j], title=strs, vmax=np.max(images[i*grid_y+j]))
plt.tight_layout(pad=.1)

In [ ]:
grid_x=5
grid_y=5
a_sampler.steps=3
samples_n, samples  = a_sampler.ancestral_sampler(grid_x*grid_y, 3)

images = samples[-1]
images = np.abs(utils.float2cplx(images))

fig, axss = plt.subplots(grid_x, grid_y, figsize=(10, 10), gridspec_kw={'width_ratios': [1  for _ in range(grid_x)]})
for i in range(grid_x):
    for j in range(grid_y):
        if i==0:
            strs='x_%d'%j
        else:
            strs=''
        axplot(axss[i,j], images[i*grid_y+j], title=strs, vmax=np.max(images[i*grid_y+j]))
plt.tight_layout(pad=.1)